In [ ]:
import os
import gc
import cv2
import time
import random

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

# Utils
from tqdm.auto import tqdm

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2
from pytorch_toolbelt import losses as L
import warnings
warnings.filterwarnings("ignore")

## using gpu:1
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything()

In [ ]:
class Customize_Model(nn.Module):
    def __init__(self, cls):
        super().__init__()
        model_name = 'csn_r101'
        self.model = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True)
        self.model.blocks[-1].proj= nn.Linear(in_features=2048, out_features=cls, bias=True)
#         self.model.projection.model= nn.Linear(in_features=2048, out_features=cls, bias=True)
        
    def forward(self, image):
#         x= nn.functional.interpolate(image, 
#                                      size=(32,CFG['img_size'],CFG['img_size']), 
#                                      scale_factor=None, 
#                                      mode='nearest', 
#                                      align_corners=None)
        x = self.model(image)
        return x
    
# torch.hub.list('facebookresearch/pytorchvideo')
# model= Customize_Model(2)
# x= torch.rand(1,3,16,512,512)
# x= model(x)
# x.shape

In [ ]:
def get_train_transform(img_size):
    return A.Compose([
        A.Resize(img_size, img_size),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        A.HorizontalFlip(p=0.5),
#         A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.15, rotate_limit= 10,
                                        interpolation=cv2.INTER_LINEAR, border_mode=0, p=0.7),
        ToTensorV2(p=1.0),
    ])


def get_test_transform(img_size):
    return A.Compose([
        A.Resize(img_size, img_size),
        ToTensorV2(p=1.0),
    ])

def get_resize_transform(height, width):
    return A.Compose([
        A.Resize(width, height, p=1),
    ] )

In [ ]:
def read_video(path, 
               hitframe,
               is_train,
               frame_length=30,
               video_load_frac=1.0):
    imgs= []
    cap= cv2.VideoCapture(path)
    while cap.isOpened():
        ret, img = cap.read()
        if not ret: break
        img= cv2.resize(img, None, fx=video_load_frac, fy=video_load_frac)
        img= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        imgs.append(img)
#         if len(imgs)>150: break
    
    ## make label
    label = [0]*len(imgs)
    for i in hitframe: 
        try:
            label[i-3]= 1
            label[i-2]= 1
            label[i-1]= 1
            label[i]= 1
            label[i+1]= 1
        except:
            break
        
    ## clip video
    if is_train:
        while True:
            clip_indx= np.random.randint(len(imgs)-frame_length+1) if (len(imgs)-frame_length)>=0 else 0
            clip_imgs= imgs[clip_indx:clip_indx+frame_length]
            clip_imgs= np.array(clip_imgs)
            clip_label= label[clip_indx:clip_indx+frame_length]
            if np.sum(clip_label)>1: break
        imgs, label= clip_imgs, clip_label
    
    return np.array(imgs), np.array(label) ## (img_len, H, W)

def read_label(path):
    df= pd.read_csv(path)
    return df['HitFrame'].values

class Customize_Dataset(Dataset):
    def __init__(self, df, transforms=None, is_train=True):
        self.df = df
        self.transforms = transforms
        self.is_train= is_train
    
    def __getitem__(self, index):
        data = self.df.loc[index]
        hitframe= read_label(data['label'])
        img, label = read_video(data['image_path'],
                               hitframe,
                               self.is_train,
                               frame_length= CFG['frame_length'])
        img = img.transpose(1,2,0)
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
        
        ## convert to 3 channel
        img= img.unsqueeze(dim=0)
        img= img.expand(3,img.shape[1],img.shape[2],img.shape[3])
        
        return {
            'image': torch.tensor(img/255, dtype=torch.float32),
            'label': torch.tensor(label, dtype=torch.float32),
        }
    
    def __len__(self):
        return len(self.df)

In [ ]:
class Customize_loss(nn.Module):
    def  __init__(self):
        super().__init__()
        self.soft_bce= L.SoftBCEWithLogitsLoss(smooth_factor= 0.01)
    
    def forward(self, y_pred, y_true):
        loss= 1*self.soft_bce(y_pred, y_true)
        return loss

In [ ]:
def train_epoch(dataloader, model, criterion, optimizer):
    scaler= amp.GradScaler()
    model.train()

    ep_loss= []
    for i, data in enumerate(tqdm(dataloader)):

        imgs= data['image'].to('cuda')
        labels= data['label'].to('cuda')
        
        with amp.autocast():
            preds= model(imgs)
            loss= criterion(preds, labels)
            ep_loss.append(loss.item())
            loss/= CFG['gradient_accumulation']
            scaler.scale(loss).backward()
            
            if (i+1) % CFG['gradient_accumulation']== 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                
    return np.mean(ep_loss)

In [ ]:
from metrics import *

def inference(model, img):
    for i, m in enumerate(model):
        with torch.no_grad():
            m.eval()
            pred= m(img)[0]
                
        if i==0: preds= pred.sigmoid()
        else: preds+= pred.sigmoid()
            
    pred= preds/len(model)
    pred= pred.cpu().numpy().tolist()
    return pred

def valid_epoch(dataloader, model, criterion):
    model.eval()
    
    ep_loss= []
    all_pred= []
    all_label= []
    for i, data in enumerate(tqdm(dataloader)):

        imgs= data['image']
        labels= data['label'].to('cuda')[0]
        all_label.extend(labels.cpu().numpy())
        
        total_pred= []
        indx=0
        for f in range(imgs.shape[2]):
            img= imgs[:,:,indx:indx+CFG['frame_length']]
            if img.shape[2]<CFG['frame_length']: break
            pred= inference([model], img.to('cuda'))
            total_pred+= pred
            indx+= CFG['frame_length']
            
        ## fill total_pred length to label
        total_pred+= [0]* (len(labels)-len(total_pred))
        all_pred+= total_pred
        
    
    ## caculate metrics
    all_label= np.array([all_label])
    all_label= np.where(all_label>0.5, 1, 0)
    all_pred= np.array([all_pred])
    
    auc= AUC(all_pred, all_label)
    print(f'AUC: {auc}')
    
    score= auc
    return np.mean(ep_loss), score

# CFG

In [ ]:
CFG= {
    'fold': 4,
    'epoch': 500,
    
    'img_size': 640,
    'frame_length': 32,
    
    'batch_size': 1,
    'gradient_accumulation': 1,
    
    'lr': 1e-5,
    'weight_decay': 1e-4,
    
    'load_model': './test_model/hitframe/csn_s640_d32/cv4_best.pth',
    'save_model': './train_model'
}
CFG

# Prepare Dataset

In [ ]:
df= pd.read_csv('data/train.csv')
train_dataset= df[df['fold']!=CFG['fold']].reset_index(drop=True)
valid_dataset= df[df['fold']==CFG['fold']].reset_index(drop=True)
print(f'train dataset: {len(train_dataset)}')
print(f'valid dataset: {len(valid_dataset)}')

train_dataset= Customize_Dataset(train_dataset.iloc[:], get_train_transform(CFG['img_size']), is_train=True)
valid_dataset= Customize_Dataset(valid_dataset.iloc[:], get_test_transform(CFG['img_size']), is_train=False)

train_loader= DataLoader(train_dataset, batch_size= CFG['batch_size'], shuffle=True, num_workers=0)
valid_loader= DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=0)
df.head()

# Train

In [ ]:
## create model
if CFG['load_model']:
    print(f"load_model: {CFG['load_model']}")
    model= torch.load(CFG['load_model'], map_location= 'cuda')
else:
    model= Customize_Model(CFG['frame_length'])
model.to('cuda')
    
## hyperparameter
criterion= Customize_loss()
optimizer= optim.AdamW(model.parameters(), lr= CFG['lr'], weight_decay= CFG['weight_decay'])

## start training
best_score= 0
for ep in range(1, CFG['epoch']+1):
    print(f'\nep: {ep}')
    
    train_loss= train_epoch(train_loader, model, criterion, optimizer)
    print(f'train loss: {round(train_loss, 5)}')
    
    valid_loss, valid_acc= valid_epoch(valid_loader, model, criterion)
    print(f'valid loss: {round(valid_loss, 5)}, valid_acc: {round(valid_acc, 5)}')
    
    if valid_acc >= best_score:
        best_score= valid_acc
        torch.save(model, f"{CFG['save_model']}/cv{CFG['fold']}_best.pth")
        print(f'model save at score: {round(best_score, 5)}')
        
    ## save model every epoch
#     torch.save(model, f"{CFG['save_model']}/cv{CFG['fold']}_ep{ep}.pth")